# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import pprint as pprint
# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-7.15,63,75,3.09,CA,1698590863
1,1,albany,42.6001,-73.9662,7.35,91,100,2.43,US,1698590641
2,2,grytviken,-54.2811,-36.5092,3.22,88,48,1.98,GS,1698590865
3,3,jamestown,42.0970,-79.2353,7.51,98,100,4.12,US,1698590664
4,4,ribeira grande,38.5167,-28.7000,18.22,63,75,5.14,PT,1698590866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = ("Humidity"),
    scale = 0.80,
    color = "City"
)


# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values

df1 = city_data_df[city_data_df["Max Temp"].between(21,27)]
df2 = df1[df1["Wind Speed"]<4]
df3 = df2[df2["Cloudiness"] <=10]


# Drop any rows with null values


# Display sample data
df3


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,haiku-pauwela,20.9219,-156.3051,24.47,78,0,0.00,US,1698590878
28,28,fort bragg,35.1390,-79.0060,23.79,74,0,3.60,US,1698590847
44,44,punahana,27.8667,77.2000,26.78,33,0,2.34,IN,1698590890
50,50,katsuren-haebaru,26.3369,127.8719,23.01,82,0,3.09,JP,1698590893
62,62,pisco,-13.7000,-76.2167,22.79,73,0,3.60,PE,1698590902
76,76,hilo,19.7297,-155.0900,22.36,79,0,3.60,US,1698590518
81,81,bolintin vale,44.4489,25.7578,23.86,39,0,1.92,RO,1698590914
91,91,hawaiian paradise park,19.5933,-154.9731,22.24,94,0,3.60,US,1698590919
101,101,al ghayzah,16.2079,52.1760,26.60,68,9,3.78,YE,1698590924
119,119,singaraja,-8.1120,115.0882,26.24,76,7,2.75,ID,1698590936


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df3[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,haiku-pauwela,US,20.9219,-156.3051,78,
28,fort bragg,US,35.1390,-79.0060,74,
44,punahana,IN,27.8667,77.2000,33,
50,katsuren-haebaru,JP,26.3369,127.8719,82,
62,pisco,PE,-13.7000,-76.2167,73,
76,hilo,US,19.7297,-155.0900,79,
81,bolintin vale,RO,44.4489,25.7578,39,
91,hawaiian paradise park,US,19.5933,-154.9731,94,
101,al ghayzah,YE,16.2079,52.1760,68,
119,singaraja,ID,-8.1120,115.0882,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 1
params = {
    "categories": categories,
    "apiKey":geoapify_key,
    "limit": limit,   
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
fort bragg - nearest hotel: Airborne Inn Lodging
punahana - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
hilo - nearest hotel: Dolphin Bay Hotel
bolintin vale - nearest hotel: Motel Monrovia
hawaiian paradise park - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
singaraja - nearest hotel: Hotel Sentral
ahiri - nearest hotel: No hotel found
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
constantia - nearest hotel: Hotel Chérica
alegrete - nearest hotel: Expresso Hotel
banswada - nearest hotel: No hotel found
southern pines - nearest hotel: Jefferson Inn
cabo san lucas - nearest hotel: Comfort Rooms
ardestan - nearest hotel: Tourism Hotel
tan-tan - nearest hotel: Sable d´or
conduaga - nearest hotel: No hotel found
khorramabad - nearest hotel: هتل رنگین کمان
rakovski - nearest hotel: No hotel found
morehead city -

,City,Country,Lat,Lng,Humidity,Hotel Name
27,haiku-pauwela,US,20.9219,-156.3051,78,Inn At Mama's Fish House
28,fort bragg,US,35.1390,-79.0060,74,Airborne Inn Lodging
44,punahana,IN,27.8667,77.2000,33,No hotel found
50,katsuren-haebaru,JP,26.3369,127.8719,82,No hotel found
62,pisco,PE,-13.7000,-76.2167,73,La Portada
76,hilo,US,19.7297,-155.0900,79,Dolphin Bay Hotel
81,bolintin vale,RO,44.4489,25.7578,39,Motel Monrovia
91,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
101,al ghayzah,YE,16.2079,52.1760,68,فندق تاج العرب
119,singaraja,ID,-8.1120,115.0882,76,Hotel Sentral


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = ("Humidity"),
    scale = 0.80,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)


# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)